In [2]:
import torch
from src.transformers.models.mamba.configuration_mamba import MambaConfig
from src.transformers.models.mamba.modeling_mamba import MambaMixer

In [3]:
torch.manual_seed(23456)
config1 = MambaConfig(use_mambapy=False, vocab_size=60, hidden_size=64, num_hidden_layers=4)
mixer1 = MambaMixer(config1, 0)

torch.manual_seed(23456)
config2 = MambaConfig(use_mambapy=True, vocab_size=60, hidden_size=64, num_hidden_layers=4)
mixer2 = MambaMixer(config2, 0)

The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [18]:
x = torch.randn(2, 12, 64)

In [19]:
y1 = mixer1(x)
J_1 = y1.sum()
J_1.backward()

y2 = mixer2(x)
J_2 = y2.sum()
J_2.backward()

print(torch.allclose(y1, y2))

True


In [20]:
gradients_same = True
for param1, param2 in zip(mixer1.parameters(), mixer2.parameters()):
    if not torch.allclose(param1.grad, param2.grad, rtol=0.01):
        gradients_same = False
        break

print(gradients_same)

True
